# **Data Inspection and Data Cleaning**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'


In [ ]:
data = pd.read_excel(url)

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


**Data Cleaning**:


*   Check for missing data
*   Remove duplicate records data











In [ ]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


This provides a preview of outliers. Any column with a mean significantly deviating from the median (around 50%) is considered an outlier.

* **Check for missing data**




In [ ]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [ ]:
# determine data type of columns
def get_column_types(data):
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        if dtype.startswith('int') or dtype.startswith('float'):
            if (data[col] % 1 == 0).all():  # Check if all values are integers
                types.append('Discrete')
            else:
                types.append('Continuous')
        elif dtype.startswith('datetime'):
            types.append('Date')
        elif dtype.startswith('object') or dtype.startswith('category'):
            types.append('Categorical')
        else:
            types.append('Unknown')
    return types


missing_data = pd.DataFrame({
    'Data type': get_column_types(data),
    'Missing data count': data.isna().sum()
}, index=data.columns)

# highlight cells with missing data
def missing(s):
    return ['background-color: pink' if v else '' for v in s]

missing_data = missing_data.style.apply(missing, subset=['Missing data count'])
missing_data


,Data type,Missing data count
InvoiceNo,Categorical,0
StockCode,Categorical,0
Description,Categorical,1454
Quantity,Discrete,0
InvoiceDate,Date,0
UnitPrice,Continuous,0
CustomerID,Continuous,135080
Country,Categorical,0


'Description' and 'CustomerID' columns have null values.

So, check if it is possible to fill **CustomerID** from **InvoiceNo**

In [ ]:
def assessMapping(df, col1, col2):
    max_occurrences = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    print(max_occurrences == 1)

In [ ]:
# Verify if each InvoiceNo corresponds to a single CustomerID
assessMapping(data, 'InvoiceNo', 'CustomerID')

True


Since we can't fill in missing CustomerID values and we have adequate data, we will simply drop any observations that have NaN values in the CustomerID column.

In [ ]:
data.dropna(subset=['CustomerID'], inplace=True)
print(data.isna().sum())


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


Dropping NaN values in CustomerId coulmn  has removed all missing Description rows too.

* **Remove duplicate data records**

In [ ]:
data.drop_duplicates(inplace=True)
data.reset_index(drop=True)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
401599,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
401600,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
401601,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
401602,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


The transactions of the last month in year 2011 have only data for 9 days. Hence remove them too.

In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceMonth'] = data['InvoiceDate'].dt.to_period('M').dt.to_timestamp()

# Filter out transactions after November 2011
data = data[data['InvoiceMonth'] <= '2011-11']
data.reset_index(drop=True, inplace=True)
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


In [ ]:
data.to_csv('/content/gdrive/MyDrive/Masters Capstone Project/cleaned_data.csv', index=False)